In [1]:
import torch
from dataset import ru_preprocess,train_dataset, BOS_IDX,EOS_IDX,UNK_IDX,PAD_IDX,ru_vocab
from config import DEVICE,SEQ_MAX_LEN

# de翻译到en
def translate(transformer,ru_sentence):
    # Ru分词
    ru_tokens,ru_ids=ru_preprocess(ru_sentence)
    if len(ru_tokens)>SEQ_MAX_LEN:
        raise Exception('不支持超过{}的句子'.format(SEQ_MAX_LEN))

    # Encoder阶段
    enc_x_batch=torch.tensor([ru_ids],dtype=torch.long).to(DEVICE)      # 准备encoder输入
    encoder_z=transformer.encode(enc_x_batch)    # encoder编码

    # Decoder阶段
    ru_token_ids=[BOS_IDX] # 翻译结果
    while len(ru_token_ids)<SEQ_MAX_LEN:
        dec_x_batch=torch.tensor([ru_token_ids],dtype=torch.long).to(DEVICE)  # 准备decoder输入
        decoder_z=transformer.decode(dec_x_batch,encoder_z,enc_x_batch)   # decoder解碼
        next_token_probs=decoder_z[0,dec_x_batch.size(-1)-1,:]    # 序列下一个词的概率
        next_token_id=torch.argmax(next_token_probs)    # 下一个词ID
        ru_token_ids.append(next_token_id)

        if next_token_id==EOS_IDX:  # 结束符
            break

    # 生成翻译结果
    ru_token_ids=[id for id in ru_token_ids if id not in [BOS_IDX,EOS_IDX,UNK_IDX,PAD_IDX]] # 忽略特殊字符
    ru_tokens=ru_vocab.lookup_tokens(ru_token_ids)    # 词id序列转token序列
    return ' '.join(ru_tokens)


if __name__=='__main__':
    # 加载模型
    transformer=torch.load('checkpoint/model.pth')
    transformer.eval()

In [2]:
for i in range(100):
        en,ru=train_dataset[i]
        ru1=translate(transformer,en)
        print('{} -> {} -> {}'.format(en,ru,ru1))

My brother has a dog. -> У моего брата есть собака. -> Она довольно хорошая больница .
Let's not talk shop, OK? -> Не будем говорить о работе, ладно? -> Ты всегда знает , что Том занят тебя слышал ?
When Tom woke up, everything was covered with snow. -> Когда Том проснулся, всё было усыпано снегом. -> Что Вы , чтобы Том попросил Мэри об этом раньше .
Do you really think I can help you? -> Ты правда думаешь, что я могу тебе помочь? -> Что Вы собираетесь , кому занят вчера тебя слышал .
It was very hot inside the bus. -> В автобусе было очень жарко. -> Что Вы были десять долларов с тобой .
I really don't think that's necessary. -> Я правда не думаю, что это необходимо. -> Что будете Мэри о своём город .
It isn't as cold here as in Alaska. -> Здесь не так холодно, как на Аляске. -> Что Вы были десять долларов с тобой .
I think it is a gift. -> По-моему, это подарок. -> Что будете Мэри о своём город .
I'll see you next month. -> Увидимся в следующем месяце. -> Она довольно хорошая больница

In [4]:
en = 'Hi !'
ru1=translate(transformer,en)
print('{} -> {}'.format(en,ru1))

Hi ! -> Я работаю быстро


In [5]:
en = 'I love you.'
ru1=translate(transformer,en)
print('{} -> {}'.format(en,ru1))

I love you. -> Такие вещи происходят .
